## NLTK In Class Excercise

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

### Let's also add punctuation to the list of stop words because we don't want that as well.

In [25]:
import pandas as pd
import string
test_reviews = pd.read_csv("south_west_review.csv")
test_reviews['true_sentiment'] = ['positive' if x == 5 else 'negative' if x <= 2 else 'none' 
                                  for x in test_reviews.Review_Score]
stopwords_list = stopwords.words('english') + list(string.punctuation)

test_data = []
reviews = list(test_reviews.Review_Text.str.lower())
true_sentiment = list(test_reviews.true_sentiment)
for i in range(0,len(reviews)):
    word_list = word_tokenize(reviews[i])
    word_list = [word for word in word_list if word not in stopwords_list]
    word_dictionary = dict([(word, True) for word in word_list])
    sample_tuple = (word_dictionary,true_sentiment[i])
    test_data.append(sample_tuple)

In [26]:
print(len(test_reviews[test_reviews.true_sentiment == 'positive']))
print(len(test_reviews))
len(test_reviews[test_reviews.true_sentiment == 'positive'])/len(test_reviews)

851
1397


0.6091624910522548

#### Now we will conduct sentiment analysis on the test reviews using Microsoft Azure text analytics to compare.

In [ ]:
yourKey = "d74e1ee2febf4c308b16bdf2375cd30a"
import requests
import time
azure_sentiments = []
review_number = 1
time_counter = time.time()
for review in list(test_reviews.Review_Text):
    if review_number % 100 == 0:
        print("waiting")
        print(abs(75 - (time.time() - time_counter)))
        time.sleep(abs(75 - (time.time() - time_counter)))
        time_counter = time.time()
    print(review_number)
    review_number+=1
    text_analytics_base_url = "https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/"
    
    sentiment_api_url = text_analytics_base_url + "sentiment"
    
    documents = {'documents' : [{'id': '1', 'language': 'en', 'text': review}]}
    
    headers   = {"Ocp-Apim-Subscription-Key": yourKey}
    response  = requests.post(sentiment_api_url, headers=headers, json=documents)
    sentiments = response.json()
    azure_sentiments.append(sentiments['documents'][0]['score'])
    print(sentiments)
    

1
{'documents': [{'id': '1', 'score': 0.06472092866897583}], 'errors': []}
2
{'documents': [{'id': '1', 'score': 0.7985153794288635}], 'errors': []}
3
{'documents': [{'id': '1', 'score': 0.06209379434585571}], 'errors': []}
4
{'documents': [{'id': '1', 'score': 0.9068057537078857}], 'errors': []}
5
{'documents': [{'id': '1', 'score': 0.24061641097068787}], 'errors': []}
6
{'documents': [{'id': '1', 'score': 0.9894037246704102}], 'errors': []}
7
{'documents': [{'id': '1', 'score': 0.011934995651245117}], 'errors': []}
8
{'documents': [{'id': '1', 'score': 0.22159379720687866}], 'errors': []}
9
{'documents': [{'id': '1', 'score': 0.8422805070877075}], 'errors': []}
10
{'documents': [{'id': '1', 'score': 0.9517775774002075}], 'errors': []}
11
{'documents': [{'id': '1', 'score': 0.05097442865371704}], 'errors': []}
12
{'documents': [{'id': '1', 'score': 0.8634535074234009}], 'errors': []}
13
{'documents': [{'id': '1', 'score': 0.11995288729667664}], 'errors': []}
14
{'documents': [{'id': '

In [28]:
test_reviews['sentiment']=azure_sentiments

ValueError: Length of values does not match length of index

In [29]:
len(azure_sentiments)

0

In [24]:
test_reviews.to_csv('southwest.csv')

In [ ]:
azure_sentiment_rating = ["positive" if x>0.5 else "negative" for x in azure_sentiments]
azure_sentiment_rating

#### Now count how many are matched divided by the total reviews.  As we can see, the Azure text analytics clearly outperformed the Naive Bayes which we ran using Python NLTK.

In [ ]:
azure_correct = 0
for i in range(0,len(test_reviews)):
    if list(test_reviews.true_sentiment)[i] == azure_sentiment_rating[i]:
        azure_correct += 1
azure_correct

In [ ]:
azure_correct / len(test_reviews)